In [1]:
import cv2
import numpy as np
import os
from PIL import Image

In [2]:

images=os.listdir('Изображения тест')
crop_images=os.listdir('Вырезанные изображения')
for image in images:
    for crop_image in crop_images:
        if image==crop_image:
            original_image = Image.open('Изображения тест/'+image)
            cropped_image=Image.open('Вырезанные изображения/'+image)
            original_image = np.array(original_image)
            cropped_image = np.array(cropped_image)
            height, width, _= cropped_image.shape
            h, w, _=original_image.shape
            result = cv2.matchTemplate(original_image, cropped_image, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            top_left = max_loc
            bottom_right = (top_left[0] + width, top_left[1] + height)
            with open('boxes/'+image, mode='w') as txt_file:
                txt_file.write(str(top_left[0]/w)+' '+str(top_left[1]/h)+' '+str(bottom_right[0]/w)+' '+str(bottom_right[1]/h))

In [3]:
def calculate_iou(box1, box2):
    """
    box1 и box2: [x1, y1, x2, y2], где (x1, y1) — левый верхний угол, (x2, y2) — правый нижний.
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area_box1 + area_box2 - intersection
    iou = intersection / union if union != 0 else 0
    return iou

In [9]:
def calculate_precision_recall(gt_boxes, pred_boxes, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for pred_box in pred_boxes:
        matched = False
        for gt_box in gt_boxes:
            iou = calculate_iou(gt_box, pred_box)
            if iou > iou_threshold:
                true_positives += 1
                matched = True
                break
        if not matched:
            false_positives += 1
            
    false_negatives = len(gt_boxes) - true_positives
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

In [10]:
gt_boxes=[]
pred_boxes=[]
ious=[]
labels=os.listdir('Faces.v1i.yolov12/train/labels')
boxes=os.listdir('boxes')
for label in labels:
    for box in boxes:
        if label[:24]==box[:24]:
            with open('boxes/'+box) as box_file:
                content1 = box_file.read()
            with open('Faces.v1i.yolov12/train/labels/'+label) as label_file:
                content2=label_file.read()
            gt_box=content1.split('\n')[0]
            gt_box = [float(item) for item in gt_box.split()]
            gt_boxes.append(gt_box)
            for pred_box in content2.split('\n'):
                pred_box=pred_box.split()[1:]
                pred_box = [float(pred_box[0])-float(pred_box[2])/2, float(pred_box[1])-float(pred_box[3])/2, float(pred_box[0])+float(pred_box[2])/2, float(pred_box[1])+float(pred_box[3])/2]
                pred_boxes.append(pred_box)
                iou = calculate_iou(gt_box, pred_box)
                print(iou)
                ious.append(iou)
            print(box)
                
precision, recall = calculate_precision_recall(pred_boxes, gt_boxes)
print(f"Precision: {precision}, Recall: {recall}")
            

0.9755900746419206
2025-03-22-14-46-02_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.7354372123602894
0.0
2025-03-22-14-46-13_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.0
0.8471108201014914
2025-03-22-14-49-03_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.0
0.6056572379367721
2025-03-22-14-49-07_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.0
0.5399553571428586
2025-03-22-14-49-25_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.7621746972429777
0.0
2025-03-22-14-49-49_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.3834586466165416
2025-03-22-14-50-35_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.0
0.3835493965131867
2025-03-22-14-50-38_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.0
0.3722126929674101
2025-03-22-14-50-53_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.0
0.4417670682730924
2025-03-22-14-50-59_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.4390501319261212
2025-03-22-14-51-09_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.0
2025-03-22-14-51-19_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg
0.0
0.5262104113578449
2025-03-22-14-51-48_TEST_ЛЮДИ_АВТОБУ

In [6]:
summa=0
count=0
for elem in ious:
    if elem!=0:
        summa+=elem
        count+=1
print(summa/count)

0.6515559365872229


In [7]:
image='2025-03-22-14-54-39_TEST_ЛЮДИ_АВТОБУС_Определение лиц.jpg'
original_image = Image.open('Изображения тест/'+image)
original_image.show()

In [8]:
cropped_image = Image.open('Вырезанные изображения/'+image)
cropped_image.show()